In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings

#settings
warnings.filterwarnings("ignore")

In [2]:
appearances_df = pd.read_csv("../raw_data/appearances.csv")
clubs_df = pd.read_csv("../raw_data/clubs.csv")
games_df = pd.read_csv("../raw_data/games.csv")
players_df = pd.read_csv("../raw_data/players.csv")

In [3]:
def players_df_preproc(players_df):
    #add year to game valuations
    now = datetime.now()
    players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])
    players_df = players_df[players_df['date_of_birth'].isnull() == False]
    players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
    players_df['age'] = players_df['age'].round().astype(int) 

    # Calculate the contract remaining of each player
    players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
    players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
    players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 
    return players_df

In [4]:
def appearances_df_preproc(appearances_df):
    # add year to player appearances
    appearances_df['datetime']=pd.to_datetime(appearances_df['date'], format="%Y-%m-%d")
    appearances_df['year']=appearances_df['datetime'].dt.year
    appearances_df = appearances_df[(appearances_df.year > 2004 ) & (appearances_df.year < 2023 )]
    return appearances_df

In [5]:
def games_and_appearances_df_preproc(games_df,appearances_df,clubs_df):
    games_df['datetime']=pd.to_datetime(games_df['date'], format="%Y-%m-%d")
    games_df['year']=games_df['datetime'].dt.year
    games_df = games_df[(games_df.year > 2004 ) & (games_df.year < 2023 )]
    games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
    
    
    
    # here we impute the missing values with the minimum value of the corresponding league 
    # get the minimum value of the 'Domestic League' column
    min_value = clubs_df.groupby('domestic_competition_id')['total_market_value'].min()

    # fill missing values in 'Market Value' column with the minimum value of the 'Domestic League' column
    clubs_df['total_market_value'] = clubs_df.apply(lambda x: min_value[x['domestic_competition_id']] if pd.isnull(x['total_market_value']) else x['total_market_value'], axis=1)
    
    table_df = games_and_appearances_df
    
    # Join the tables on the home and away club IDs
    table_df = table_df.merge(clubs_df[['club_id', 'total_market_value']], 
                            how='left', 
                            left_on='home_club_id', 
                            right_on='club_id')

    table_df = table_df.merge(clubs_df[['club_id', 'total_market_value']], 
                            how='left', 
                            left_on='away_club_id', 
                            right_on='club_id',
                            suffixes=('_home', '_away'))

    # Rename the columns to reflect the market value data
    table_df = table_df.rename(columns={'market_value_home': 'home_club_market_value',
                                        'market_value_away': 'away_club_market_value'})

    # Drop the redundant club_id columns
    table_df = table_df.drop(['club_id_home', 'club_id_away'], axis=1)
    
    games_and_appearances_df = table_df
    
    
    return games_and_appearances_df

In [6]:
#transform home and away ----> for and against 
"""
 df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
       """


'\n df["goals_for"] = df.apply(lambda row: row[\'home_club_goals\'] if row[\'home_club_id\'] == row[\'player_club_id\'] \n            else row[\'away_club_goals\'] if row[\'away_club_id\'] == row[\'player_club_id\'] \n            else np.nan, axis=1)\n       '

In [7]:
#create a function to collate player stats
def player_stats(player_id, season, games_and_appearances_df):
    
    df = games_and_appearances_df[games_and_appearances_df['player_id'] == player_id]
    df =  df[df['season'] == season]    
    if (df.shape[0] == 0):
        Out = [(np.nan, season,0,0,0,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','season','goals','games',
                                                     'assists','minutes_played','goals_for',
                                                     'goals_against','clean_sheet',
                                                     'yellow_cards','red_cards',"value_goals_for"])
        return out_df    
    else:       
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        """
        df['value_goals_for'] = df.apply(lambda row: row['home_club_goals']*row['total_market_value_home'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals']*row['total_market_value_away'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        """
        df['value_goals_for'] = df.apply(lambda row: row['home_club_goals']*row['total_market_value_home'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals']*row['total_market_value_away'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan if pd.isnull(row['home_club_id']) or pd.isnull(row['away_club_id']) else np.nan, axis=1)

        
        
        
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
            else 0 if row['goals_against'] > 0
            else np.nan, axis=1)
        #weighing performances
        
        #((= goals_for * value_against))         goals scored must be multiplied for the valuer of the opposing team     
        
        
        # df["value_goals_against"]  ((= goals_against / value_against))     goals socred by opponent team must be divided by the value of the opposing team     
        # df['value_minutes_played]  ((= minutes_played * value_for))                      
        # df["clean_sheet_value"]    ((= clean_sheet * value_against ))     same logic as "value goal for"
       
       # look at the distribution
       
       
       
        # scaler probably robust (maybe standard)
        
        df = df.groupby(['player_id',"season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                    'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                    'goals_against' : 'sum', 'clean_sheet' : 'sum','yellow_cards':'sum','red_cards':'sum',
                                                                    'value_goals_for' : 'sum'})
        out_df = df.rename(columns={'game_id': 'games'})
        return out_df


In [8]:
# preprocessing function to return a dataframe
def preprocessing(clubs_df,players_df,games_and_appearances_df):
    
    merged_players_df=players_df.drop(['current_club_id', 'city_of_birth', 'date_of_birth','first_name', 'last_name', 'player_code', 'image_url', 'url'], axis=1)
    merged_players_df = merged_players_df.reindex(columns = merged_players_df.columns.tolist() + ['value_goals_for','club_value','squad_size','goals','goals_2022','games_2022','assists_2022','minutes_played_2022','goals_against_2022','goals_for_2022','clean_sheet_2022'])
    print("step 1 done")
    
    for player_id in merged_players_df.player_id.unique():
        club_id = players_df.current_club_id[(players_df.player_id==player_id)]
        try:
            merged_players_df.club_value[(players_df.player_id==player_id)]=int(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
        except:
            merged_players_df.club_value[(players_df.player_id==player_id)]='NaN'  
        merged_players_df.squad_size[(players_df.player_id==player_id)]=int((clubs_df.squad_size[(clubs_df.club_id==int(club_id))]))

    columns=['player_id','games_2022','minutes_played_2022','goals_2022','assists_2022','goals_against_2022',
             'goals_for_2022','clean_sheet_2022','name','position','sub_position','last_season','foot','height_in_cm','age',
             'country_of_citizenship','country_of_birth','current_club_name','club_value','squad_size','current_club_domestic_competition_id',
             'agent_name','contract_expiration_date','term_days_remaining','market_value_in_eur','highest_market_value_in_eur','value_goals_for']
    merged_players_df=merged_players_df[columns] 
    print("step 2 done")

    #iterate through players
    for index in merged_players_df.index:
        id = merged_players_df.loc[index][0]
        #name = merged_players_df.loc[index][1]
        
        season = 2022
        stats = player_stats(id, season, games_and_appearances_df)
  
        merged_players_df.at[index,'games_{}'.format(season)]= stats['games'][0]
        merged_players_df.at[index,'goals_{}'.format(season)]= stats['goals'][0]
        merged_players_df.at[index,'assists_{}'.format(season)]= stats['assists'][0]
        merged_players_df.at[index,'minutes_played_{}'.format(season)]= stats['minutes_played'][0]
        merged_players_df.at[index,'goals_for_{}'.format(season)]= stats['goals_for'][0]
        merged_players_df.at[index,'goals_against_{}'.format(season)]= stats['goals_against'][0]
        merged_players_df.at[index,'clean_sheet_{}'.format(season)]= stats['clean_sheet'][0]
        merged_players_df.at[index,'yellow_cards_{}'.format(season)]= stats['yellow_cards'][0]
        merged_players_df.at[index,'red_cards_{}'.format(season)]= stats['red_cards'][0]
        merged_players_df.at[index,'value_goals_for_{}'.format(season)]= stats['value_goals_for'][0]


    print("step 3 done")

    #drop nan
    merged_players_df1=merged_players_df.dropna(subset=['market_value_in_eur'])
    
    # convert position categories to Columns for test data
    dummies=pd.get_dummies(merged_players_df1[['position']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 
    dummies=pd.get_dummies(merged_players_df1[['sub_position']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 
    dummies=pd.get_dummies(merged_players_df1[['foot']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 

    #separate numeric columns
    drop_cols = ['player_id', 'clean_sheet_2022', 'name', 'position', 'sub_position', 'last_season',
       'foot', 'country_of_citizenship',
       'country_of_birth', 'current_club_name', 'club_value', 
       'current_club_domestic_competition_id', 'agent_name',
       'contract_expiration_date', 'highest_market_value_in_eur']
    merged_players_df1 = merged_players_df1.drop(columns=drop_cols)
    return merged_players_df1

In [9]:
# test above functions

players_df = players_df_preproc(players_df)


In [10]:
appearances_df = appearances_df_preproc(appearances_df)


In [11]:
games_and_appearances_df = games_and_appearances_df_preproc(games_df,appearances_df,clubs_df)


In [12]:
df_preprocessed = preprocessing(clubs_df,players_df,games_and_appearances_df)

step 1 done
step 2 done
step 3 done


In [13]:
df_preprocessed['market_value_in_eur']

9          125000.0
10         150000.0
11         300000.0
19         500000.0
28         300000.0
            ...    
28484      500000.0
28485      100000.0
28486     1200000.0
28487    10000000.0
28488     1800000.0
Name: market_value_in_eur, Length: 15577, dtype: float64

In [15]:
df_preprocessed.columns

Index(['games_2022', 'minutes_played_2022', 'goals_2022', 'assists_2022',
       'goals_against_2022', 'goals_for_2022', 'height_in_cm', 'age',
       'squad_size', 'term_days_remaining', 'market_value_in_eur',
       'value_goals_for', 'yellow_cards_2022', 'red_cards_2022',
       'value_goals_for_2022', 'position_Attack', 'position_Defender',
       'position_Goalkeeper', 'position_Midfield',
       'sub_position_Attacking Midfield', 'sub_position_Central Midfield',
       'sub_position_Centre-Back', 'sub_position_Centre-Forward',
       'sub_position_Defensive Midfield', 'sub_position_Left Midfield',
       'sub_position_Left Winger', 'sub_position_Left-Back',
       'sub_position_Right Midfield', 'sub_position_Right Winger',
       'sub_position_Right-Back', 'sub_position_Second Striker', 'foot_Both',
       'foot_Left', 'foot_Right'],
      dtype='object')

In [16]:
df1 = df_preprocessed[['games_2022',
       'goals_against_2022', 'goals_for_2022' , 'age', 'market_value_in_eur',
       'value_goals_for', 'yellow_cards_2022', 'red_cards_2022',
       'value_goals_for_2022', 'position_Attack']]

In [17]:
df1.head()

,games_2022,goals_against_2022,goals_for_2022,age,market_value_in_eur,value_goals_for,yellow_cards_2022,red_cards_2022,value_goals_for_2022,position_Attack
9,0.0,0.0,0.0,35,125000.0,NaN,0.0,0.0,0.0,1
10,0.0,0.0,0.0,39,150000.0,NaN,0.0,0.0,0.0,1
11,0.0,0.0,0.0,34,300000.0,NaN,0.0,0.0,0.0,1
19,0.0,0.0,0.0,36,500000.0,NaN,0.0,0.0,0.0,1
28,0.0,0.0,0.0,34,300000.0,NaN,0.0,0.0,0.0,0


In [18]:
games_and_appearances_df.columns

Index(['appearance_id', 'game_id', 'player_id', 'player_club_id',
       'player_current_club_id', 'date_x', 'player_name', 'competition_id_x',
       'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
       'datetime_x', 'year_x', 'competition_id_y', 'competition_type',
       'season', 'round', 'date_y', 'home_club_id', 'away_club_id',
       'home_club_goals', 'away_club_goals', 'aggregate', 'home_club_position',
       'away_club_position', 'club_home_name', 'club_away_name',
       'home_club_manager_name', 'away_club_manager_name', 'stadium',
       'attendance', 'referee', 'url', 'datetime_y', 'year_y',
       'total_market_value_home', 'total_market_value_away'],
      dtype='object')

In [19]:
df_preprocessed.columns

Index(['games_2022', 'minutes_played_2022', 'goals_2022', 'assists_2022',
       'goals_against_2022', 'goals_for_2022', 'height_in_cm', 'age',
       'squad_size', 'term_days_remaining', 'market_value_in_eur',
       'value_goals_for', 'yellow_cards_2022', 'red_cards_2022',
       'value_goals_for_2022', 'position_Attack', 'position_Defender',
       'position_Goalkeeper', 'position_Midfield',
       'sub_position_Attacking Midfield', 'sub_position_Central Midfield',
       'sub_position_Centre-Back', 'sub_position_Centre-Forward',
       'sub_position_Defensive Midfield', 'sub_position_Left Midfield',
       'sub_position_Left Winger', 'sub_position_Left-Back',
       'sub_position_Right Midfield', 'sub_position_Right Winger',
       'sub_position_Right-Back', 'sub_position_Second Striker', 'foot_Both',
       'foot_Left', 'foot_Right'],
      dtype='object')

In [20]:
df2 = games_and_appearances_df.drop(columns= ['appearance_id', 'game_id', 'player_id',
       'date_x', 'player_name', 'competition_id_x',
       'yellow_cards', 'red_cards',
        'year_x', 'competition_id_y', 'competition_type',
        'round', 'date_y',
        'home_club_position',
       'away_club_position', 'club_home_name', 'club_away_name',
       'home_club_manager_name', 'away_club_manager_name', 'stadium',
       'attendance', 'referee', 'url', 'datetime_y', 'year_y'])

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103442 entries, 0 to 1103441
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   player_club_id           1103442 non-null  int64         
 1   player_current_club_id   1103442 non-null  int64         
 2   goals                    1103442 non-null  int64         
 3   assists                  1103442 non-null  int64         
 4   minutes_played           1103442 non-null  int64         
 5   datetime_x               1103442 non-null  datetime64[ns]
 6   season                   1103442 non-null  int64         
 7   home_club_id             1103442 non-null  int64         
 8   away_club_id             1103442 non-null  int64         
 9   home_club_goals          1103442 non-null  int64         
 10  away_club_goals          1103442 non-null  int64         
 11  aggregate                1103442 non-null  object        
 12  

In [22]:
df_preprocessed.columns

Index(['games_2022', 'minutes_played_2022', 'goals_2022', 'assists_2022',
       'goals_against_2022', 'goals_for_2022', 'height_in_cm', 'age',
       'squad_size', 'term_days_remaining', 'market_value_in_eur',
       'value_goals_for', 'yellow_cards_2022', 'red_cards_2022',
       'value_goals_for_2022', 'position_Attack', 'position_Defender',
       'position_Goalkeeper', 'position_Midfield',
       'sub_position_Attacking Midfield', 'sub_position_Central Midfield',
       'sub_position_Centre-Back', 'sub_position_Centre-Forward',
       'sub_position_Defensive Midfield', 'sub_position_Left Midfield',
       'sub_position_Left Winger', 'sub_position_Left-Back',
       'sub_position_Right Midfield', 'sub_position_Right Winger',
       'sub_position_Right-Back', 'sub_position_Second Striker', 'foot_Both',
       'foot_Left', 'foot_Right'],
      dtype='object')